In [1]:
# imports.py
from support import *

import time
from datetime import datetime
from itertools import product
from math import erf, sqrt
import sys

# start the ib loop
util.startLoop()

# connect to the market set in variables.json
try:
    if not ib.isConnected():
        ib = get_connected(market, 'live')
except Exception as e:
    ib = get_connected(market, 'live')

# from json
a = assign_var(market) + assign_var('common')
for v in a:
    exec(v)

# reset the log
with open(logpath+'test.log', 'w'):
    pass # clear the run log

util.logToFile(logpath+'test.log')
    
jup_disp_adjust() # adjust jupyter's display

In [2]:
# get pnl data (asyncio)
async def pnlcoro(ib):
    '''Gets the pnl object'''
    acct = ib.managedAccounts()[0]
    pnl = ib.reqPnL(acct)
    await ib.pnlEvent
    return pnl

# pnl object raises AssertionError if run the second time!!
try:
    pnl = ib.run(pnlcoro(ib))
except AssertionError:
    pass

# convert to dictionary and remove modelcode
pnldict = pnl.dict()
del pnldict['modelCode']

acsum = get_acc_summary(ib)

# merge the dictionaries
keydata = {**pnldict, **acsum}

In [3]:
keydata

{'account': 'U8898867',
 'dailyPnL': -15379.198610000009,
 'unrealizedPnL': -31036.951556499982,
 'realizedPnL': 34.5495,
 'NLV': 607800.34,
 'initmargin': 498420.49,
 'unrealzPnL': 0.0,
 'realzPnL': 0.0,
 'avFunds': 109379.85}

In [4]:
get_acc_summary(ib)

{'NLV': 607800.34,
 'initmargin': 498420.49,
 'unrealzPnL': 0.0,
 'realzPnL': 0.0,
 'avFunds': 109379.85}

In [ ]:
get_df_buys(ib, market, prec)

In [ ]:
trades = ib.trades()
all_trades_df = util.df(t.contract for t in trades).join(util.df(t.orderStatus for t in trades)).join(util.df(t.order for t in trades), lsuffix='_')
all_trades_df.rename({'lastTradeDateOrContractMonth': 'expiry'}, axis='columns', inplace=True)
trades_cols = ['conId', 'symbol', 'localSymbol', 'secType', 'expiry', 'strike', 'right', 
               'orderId', 'permId', 'action', 'totalQuantity', 'lmtPrice', 'status']
trades_df = all_trades_df[trades_cols]
fills_df = trades_df[trades_df.status == 'Filled']

In [ ]:
# get the template
    df_opentrades = pd.read_pickle('./templates/df_opentrades.pkl')
    df_opentrades.drop(df_opentrades.index, inplace=True) # empty it!

    # get the trades
    trades = ib.openTrades()
    trade_cols = ['secType', 'conId', 'symbol', 'lastTradeDateOrContractMonth', 'strike', 'right', 'action', 
                  'status', 'orderId', 'permId', 'lmtPrice', 'filled', 'remaining']

    # append it, if available
    if trades:
        df_ot = df_opentrades.append(util.df(t.contract for t in trades).join(util.df(t.order for t in trades)).join(util.df(t.orderStatus for t in trades), lsuffix='_'))
        df_ot = df_ot[trade_cols].rename(columns={'lastTradeDateOrContractMonth': 'expiry'})
#         active_status = {'ApiPending', 'PendingSubmit', 'PreSubmitted', 'Submitted'}
        df_activebuys = df_ot[(df_ot.action == 'BUY') & (df_ot.status.isin(active_status))]
        df_activesells = df_ot[(df_ot.action == 'SELL') & (df_ot.status.isin(active_status))]
    else:
        df_activebuys = df_opentrades[trade_cols].rename(columns={'lastTradeDateOrContractMonth': 'expiry'})
        df_activesells = df_opentrades[trade_cols].rename(columns={'lastTradeDateOrContractMonth': 'expiry'})

    #... get the portfolio
    df_pfolio = portf(ib)

    # remove from portfolio options with active buys
    df_buys = df_pfolio[~df_pfolio.conId.isin(df_activebuys.conId)]

    # remove stocks, keep only options
    df_buys = df_buys[df_buys.secType == 'OPT'] 

    # Remove the opts with position > 0. These are errors / longs that should not be re-bought automatically!
    df_buys = df_buys[df_buys.position < 0]

    # Rename the column conId to optId
    df_buys = df_buys.rename(columns={'conId': 'optId'})

    # get the dte
    df_buys = df_buys.assign(dte=df_buys.expiry.apply(get_dte))

    df_buys = df_buys[df_buys.dte > 1] # spare the last day!

    #... get the expected price to sell.
    df_buys = df_buys.assign(expPrice = np.maximum( \
                                            np.minimum(df_buys.dte.apply(hvstPricePct)*df_buys.averageCost, (df_buys.marketPrice-2*prec)), \
                                            prec) \
                                            .apply(lambda x: get_prec(x, prec)))

    # set the quantity
    df_buys = df_buys.assign(qty = df_buys.position.apply(abs))
    df_buys = df_buys.assign(lot=1) # doesn't need to be lotsize for NSE

In [ ]:
class GlobalWealth(object):
    def __init__(self):
        self._global_wealth = 10.0
        self._observers = []

    @property
    def global_wealth(self):
        return self._global_wealth

    @global_wealth.setter
    def global_wealth(self, value):
        self._global_wealth = value
        for callback in self._observers:
            print('announcing change')
            callback(self._global_wealth)

    def bind_to(self, callback):
        print('bound')
        self._observers.append(callback)


class Person(object):
    def __init__(self, data):
        self.wealth = 1.0
        self.data = data
        self.data.bind_to(self.update_how_happy)
        self.happiness = self.wealth / self.data.global_wealth

    def update_how_happy(self, global_wealth):
        self.happiness = self.wealth / global_wealth


if __name__ == '__main__':
    data = GlobalWealth()
    p = Person(data)
    print(p.happiness)
    data.global_wealth = 1.0
    print(p.happiness)


In [ ]:
len(df_ot)

In [ ]:
len(df_activebuys)

In [ ]:
len(df_activesells)

In [ ]:
import wx

class MyFrame(wx.Frame):
    def __init__(self, parent, id, title):
        wx.Frame.__init__(self, parent, id, title, size=(300,300))

        panel = wx.Panel(self, -1)
        wx.Button(panel, 1, 'Show Custom Dialog', (100,100))
        self.Bind (wx.EVT_BUTTON, self.OnShowCustomDialog, id=1)

        wx.CallLater(1000, self.OnShowCustomDialog)

    def OnShowCustomDialog(self, event=None):
        dia = MyDialog(self, -1, 'buttons')
        dia.Show()
        wx.CallLater(4000, dia.Destroy)

class MyApp(wx.App):
    def OnInit(self):
        frame = MyFrame(None, -1, 'customdialog1.py')
        frame.Show(True)
        frame.Centre()
        return True

app = MyApp(0)
app.MainLoop()